In [1]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from time import sleep
from login import EMAIL
from login import PASSWORD
import re

In [2]:
driver = webdriver.Firefox()
wait = WebDriverWait(driver, 10)
driver.maximize_window()

driver.get("https://pro.urbanize.city/users/sign_in")

wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))


iframes = driver.find_elements(By.TAG_NAME, "iframe")
if iframes:
    driver.switch_to.frame(iframes[0])

email_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='email'], input[type='email']")))
email_input.send_keys(EMAIL)

password_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='password'], input[type='password']")))
password_input.send_keys(PASSWORD)

login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.prime[actionlogin]")))
login_button.click()


wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

driver.switch_to.default_content()
wait.until(EC.visibility_of_element_located((By.XPATH, "//a[contains(@class, 'border-cyan-500') and contains(text(), 'Dashboard')]")))


<selenium.webdriver.remote.webelement.WebElement (session="a4c2f485-258b-4f9a-901c-f1ce43523eb3", element="48a14e06-0e9e-4a30-a0ad-2a6c632177bf")>

In [3]:
driver.get("https://pro.urbanize.city/los_angeles/projects")

project_links = driver.find_elements(By.CSS_SELECTOR, "ul.grid a[href^='/los_angeles/projects/']")
page_urls = [link.get_attribute('href') for link in project_links]
project_data = {url: {} for url in page_urls}


for url in page_urls:
  driver.get(url)

  wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
  
  project_name = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1.text-5xl.font-semibold.text-gray-900.tracking-tight.mb-2"))).text
  project_data[url]['project_name'] = project_name

  grid_exists = len(driver.find_elements(By.CSS_SELECTOR, "ul.grid")) > 0

  if grid_exists:
      grid_links = driver.find_elements(By.CSS_SELECTOR, "ul.grid li a")
      grid_urls = [link.get_attribute('href') for link in grid_links]
      project_data[url]['grid_urls'] = grid_urls
      for article_url in grid_urls:
        driver.get(article_url)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

      



In [4]:
# def get_project_status(driver):

#     return "Status not found"

In [9]:
def pull_data(url):
  driver.get(url)

  #get project name
  wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
  project_name = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1.text-5xl.font-semibold.text-gray-900.tracking-tight.mb-2"))).text

  print(f"Project Name: {project_name}")
  

  #get project status
  status = None
  status_elements = driver.find_elements(By.CSS_SELECTOR, "div.flex.space-x-1.w-full > div")
  for element in status_elements:
    bg_div = element.find_element(By.CSS_SELECTOR, "div[class*='w-full'][class*='h-2.5']")
    if "bg-cyan-600" in bg_div.get_attribute("class"):
      status = element.find_element(By.CSS_SELECTOR, "div.text-sm").text

  print(f"Project Status: {status}")


  #get location
  location_header = driver.find_element(By.XPATH, "//h3[contains(@class, 'text-xl') and contains(text(), 'Location')]")
  location_html = location_header.find_element(By.XPATH, "following-sibling::p").get_attribute('innerHTML')
  # location_text = location_element2.text.replace('\n', ', ')
  location_text = re.sub('<[^<]+?>', '', location_html).replace('\n', '').strip()
  # location_text = ', '.join(part.strip() for part in location_text.split(','))

  print(f"Project Location: {location_text}")


  # #get companies
  # architects_div = driver.find_element(By.CSS_SELECTOR, "div.space-y-1.grow")
  # architect_elements = architects_div.find_elements(By.CSS_SELECTOR, "div.text-cyan-600.font-semibold a")
  # architects = [element.text for element in architect_elements]

  # print(f"Architects: {architects}")


  # architects_header = driver.find_element(By.XPATH, "//div[contains(@class, 'text-slate-900') and contains(., 'Architects')]")
  # architects_div = architects_header.find_element(By.XPATH, "following-sibling::div[contains(@class, 'space-y-1')]")
  # architects_elements = architects_div.find_elements(By.CSS_SELECTOR, "div.text-cyan-600.font-semibold a")
  # architects = [element.text for element in architects_elements]
  # print(f"Architects: {architects}")

  # developers_header = driver.find_element(By.XPATH, "//div[contains(@class, 'text-slate-900') and contains(., 'Developer')]")
  # developers_div = developers_header.find_element(By.XPATH, "following-sibling::div[contains(@class, 'space-y-1')]")
  # developers_elements = developers_div.find_elements(By.CSS_SELECTOR, "div.text-cyan-600.font-semibold a")
  # developers = [element.text for element in developers_elements]
  # print(f"Developers: {developers}")

  companies = {}
  try:
      companies_div = driver.find_element(By.XPATH, "//h3[contains(@class, 'text-xl') and contains(text(), 'Companies')]/ancestor::div[contains(@class, 'project-companies')]")
      company_items = companies_div.find_elements(By.XPATH, ".//li[contains(@class, 'py-2')]")
      
      for item in company_items:
          label = item.find_element(By.XPATH, ".//div[contains(@class, 'text-slate-900') and contains(@class, 'font-medium')]").text
          value_div = item.find_element(By.XPATH, ".//div[contains(@class, 'space-y-1')]")
          company_elements = value_div.find_elements(By.CSS_SELECTOR, "div.text-cyan-600.font-semibold a")
          companies[label] = [element.text for element in company_elements]
          print(f"{label}: {', '.join(companies[label])}")
  except:
      print("Companies section not found or could not be parsed")

  project_info = {}
  try:
        information_div = driver.find_element(By.XPATH, "//h3[contains(@class, 'text-xl') and contains(text(), 'Information')]/ancestor::div[contains(@class, '')]")
        info_items = information_div.find_elements(By.XPATH, ".//li[contains(@class, 'py-4')]")
        
        for item in info_items:
            label = item.find_element(By.XPATH, ".//div[contains(@class, 'text-slate-900') and contains(@class, 'font-medium')]").text
            value_div = item.find_element(By.XPATH, ".//div[contains(@class, 'text-slate-800') and contains(@class, 'font-normal')]")
            
            if label == "Project uses":
                uses = [use.text for use in value_div.find_elements(By.XPATH, ".//div/div")]
                project_info[label] = uses
                print(f"Project uses: {', '.join(uses)}")
            else:
                value = value_div.text.strip()
                project_info[label] = value
                print(f"{label}: {value}")
  except:
        print("Information section not found or could not be parsed")

    # Return the extracted data as a dictionary
  result = {
        "project_name": project_name,
        "status": status,
        "location": location_text,
        **companies,
        **project_info
    }
    
  print(result)





# pull_data('https://pro.urbanize.city/los_angeles/projects/d506b9d2/2143-e-violet-street')
pull_data('https://pro.urbanize.city/los_angeles/projects/7018f2cb/st-ambrose-senior-housing')

Project Name: St Ambrose Senior Housing
Project Status: Proposed
Project Location: 830 Bonita Avenue    Glendora, CA 91740
Developer: St. Ambrose Episcopal Church
Project uses: Apartments
Podium: No
# of apartments: 59
Construction type: Type 3 4 5
{'project_name': 'St Ambrose Senior Housing', 'status': 'Proposed', 'location': '830 Bonita Avenue    Glendora, CA 91740', 'Developer': ['St. Ambrose Episcopal Church'], 'Project uses': ['Apartments'], 'Podium': 'No', '# of apartments': '59', 'Construction type': 'Type 3 4 5'}


In [ ]:
def pull_article_data(url):
  driver.get(url)

  wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))



In [ ]:
project_data